In [1]:
from cosapp.base import System, Port, BaseConnector
import numpy as np
from cosapp.drivers import RungeKutta, NonLinearSolver
import plotly.graph_objs as go
from cosapp.recorders import DataFrameRecorder
import plotly.express as px
import matplotlib.pyplot as plt

# Rotation Matrix

In [2]:
def RotMat3D(ang):
    
    #ang[0] = phi, ang[1] = theta, ang[2] = psi
    
    l1 = np.array([np.cos(ang[2])*np.cos(ang[1]), -np.sin(ang[2])*np.cos(ang[0]) + np.cos(ang[2])*np.sin(ang[1])*np.sin(ang[0]),
                    np.sin(ang[2])*np.sin(ang[0]) + np.cos(ang[2])*np.sin(ang[1])*np.cos(ang[0])])
    l2 = np.array([np.sin(ang[2])*np.cos(ang[1]),  np.cos(ang[2])*np.cos(ang[0]) + np.sin(ang[2])*np.sin(ang[1])*np.sin(ang[0]),
                   -np.cos(ang[2])*np.sin(ang[0]) + np.sin(ang[2])*np.sin(ang[1])*np.cos(ang[0])])
    l3 = np.array([-np.sin(ang[1]), np.cos(ang[1])*np.sin(ang[0]), np.cos(ang[1])*np.cos(ang[0])])
    
    A = np.round(np.array([l1,l2,l3]), 5)
    
    return A

# Ports

In [3]:
class PosPort(Port):
    """Position Port """
    def setup(self):
        self.add_variable('val', np.zeros(3), desc = "Velocity Components", unit = 'm')
        #self.add_variable('ang', np.zeros(3), desc = "Axes' Euler's Angles", unit = '')
        

    class Connector(BaseConnector):
        """Custom connector for `PosPort` objects
        """
        def __init__(self, name: str, sink: Port, source: Port, *args, **kwargs):
            super().__init__(name, sink, source)

        def transfer(self) -> None:
            sink = self.sink
            source = self.source
            
            if sink.owner.parent == source.owner:
                sink.val = np.matmul(RotMat3D(sink.owner[f'{sink.owner.name}_ang']), source.val)

            elif sink.owner == source.owner.parent:
                sink.val = np.matmul(RotMat3D(-source.owner[f'{source.owner.name}_ang']), source.val)
                
            else:
                sink.val = source.val

In [4]:
class VelPort(Port):
    """Velocity Port """
    def setup(self):
        self.add_variable('val', np.zeros(3), desc = "Velocity Components", unit = 'm/s')
        #self.add_variable('ang', np.zeros(3), desc = "Axes' Euler's Angles", unit = '')
        

    class Connector(BaseConnector):
        """Custom connector for `VelPort` objects
        """
        def __init__(self, name: str, sink: Port, source: Port, *args, **kwargs):
            super().__init__(name, sink, source)

        def transfer(self) -> None:
            sink = self.sink
            source = self.source
            
            if sink.owner.parent == source.owner:
                sink.val = np.matmul(RotMat3D(sink.owner[f'{sink.owner.name}_ang']), source.val)

            elif sink.owner == source.owner.parent:
                sink.val = np.matmul(RotMat3D(-source.owner[f'{source.owner.name}_ang']), source.val)
                
            else:
                sink.val = source.val

In [5]:
class AclPort(Port):
    """Acceleration Port """
    def setup(self):
        self.add_variable('val', np.zeros(3), desc = "Acceleration Components", unit = 'm/s**2')
        #self.add_variable('ang', np.zeros(3), desc = "Axes' Euler's Angles", unit = '')
        

    class Connector(BaseConnector):
        """Custom connector for `AclPort` objects
        """
        def __init__(self, name: str, sink: Port, source: Port, *args, **kwargs):
            super().__init__(name, sink, source)

        def transfer(self) -> None:
            sink = self.sink
            source = self.source
            
            if sink.owner.parent == source.owner:
                sink.val = np.matmul(RotMat3D(sink.owner[f'{sink.owner.name}_ang']), source.val)

            elif sink.owner == source.owner.parent:
                sink.val = np.matmul(RotMat3D(-source.owner[f'{source.owner.name}_ang']), source.val)
                
            else:
                sink.val = source.val

# Global System

In [6]:
class Earth(System):
    
    def setup(self):
        
        #Earth children
        self.add_child(Rocket('Rocket'))
        self.add_child(Trajectory('Traj'))
        self.add_child(Gravity('Grav'))
        self.add_child(Atmosphere('Atmo'))
        
        self.connect(self.Rocket, self.Traj, {'v_out.val' : 'v'})
        self.connect(self.Rocket, self.Grav, ['g'])
        self.connect(self.Rocket, self.Atmo, ['P', 'V_wind', 'rho'])
        self.connect(self.Grav, self.Traj, {'r_in' : 'r_out'})
        self.connect(self.Atmo, self.Traj, {'r_in' : 'r_out'})
        
        #Execution order
        self.exec_order = ['Grav', 'Atmo', 'Rocket', 'Traj']
        
        

## Earth Children

Vectors are given in the Earth-fixed reference frame

In [7]:
class Rocket(System):
    
    def setup(self):
        
        #System orientation
        self.add_inward('Rocket_ang', np.zeros(3), desc = "Earth Euler Angles", unit = '')
        
        #Engine propulsive debt
        self.add_inward('qp', 10., desc = "Engine's Propulsive Debt", unit = 'kg/s')
        
        #Rocket children
        self.add_child(Kinematics('Kin'), pulling = ['v_out'])
        self.add_child(Dynamics('Dyn'), pulling = ['g'])
        self.add_child(Thrust('Thrust'), pulling = ['P', 'qp'])
        self.add_child(Geometry('Geom'), pulling = ['qp'])
        self.add_child(Aerodynamics('Aero'), pulling = ['V_wind', 'rho'])
        
        #Child-Child connections
        self.connect(self.Kin, self.Dyn, {'Kin_ang' : 'Dyn_ang', 'v_out' : 'v_in', 'av_out' : 'av_in', 'a': 'a', 'aa' : 'aa'})
        self.connect(self.Kin, self.Aero, {'v_out' : 'v_in', 'Kin_ang' : 'Aero_ang'})
        self.connect(self.Dyn, self.Thrust, ['Fp', 'Mp'])
        self.connect(self.Dyn, self.Geom, {'m' : 'm_out', 'I' : 'I'})
        self.connect(self.Dyn, self.Aero, ['Fa', 'Ma'])
        self.connect(self.Geom, self.Aero, ['S_ref', 'gc'])
        
        #Execution order
        self.exec_order = ['Geom', 'Thrust', 'Aero', 'Dyn', 'Kin']


In [8]:
class Trajectory(System):
    
    def setup(self):
        
        #Rocket inputs
        self.add_inward('v', np.zeros(3), desc = "Rocket Velocity", unit = 'm/s')
        
        #Trajectory transients
        self.add_transient('r', der = 'v', desc = "Rocket Position")
        
        #Trajectory outputs
        self.add_outward('r_out', np.zeros(3), desc = "Rocket Position", unit = 'm')
        
    def compute(self):
        
        self.r_out = self.r
        

In [9]:
class Gravity(System):
    
    def setup(self):
        
        #System constants
        self.add_inward('G', 6.6743*10**(-11), desc = "Gravitational Constant", unit = 'N*m**2/kg**2')
        self.add_inward('M', 5.972*10**24, desc = "Earth's Mass", unit = 'kg')
        self.add_inward('R', 6.371*10**6, desc = "Earth's Radius", unit = 'm')
        
        #Trajectory inputs
        self.add_inward('r_in', np.zeros(3), desc = "Rocket Position", unit = 'm')
        
        #Gravity outputs
        self.add_output(AclPort, 'g')
        
    def compute(self):
        
        self.g.val = np.array([0., 0., -self.G*self.M/(self.R + self.r_in[-1])**2])


In [10]:
class Atmosphere(System):
    
    def setup(self):
        
        #System orientation
        self.add_inward('Atmo_ang', np.zeros(3), desc = "Earth Euler Angles", unit = '')
        
        #Atmosphere children
        self.add_child(Density('Dens'), pulling = ['r_in', 'rho'])
        self.add_child(Pressure('Pres'), pulling = ['r_in', 'P'])
        self.add_child(Wind('Wind'), pulling = ['V_wind'])
        
        #Execution order
        self.exec_order = ['Dens', 'Pres', 'Wind']
    
    

## Rocket Children

Vectors are given in the rocket-fixed reference frame

In [11]:
class Kinematics(System):
    
    def setup(self):
        
        #System orientation
        self.add_outward('Kin_ang', np.zeros(3), desc = "Rocket Euler Angles", unit = '')
        
        #Dynamics inputs
        self.add_inward('a', np.zeros(3), desc = "Rocket Acceleration", unit = 'm/s**2')
        self.add_inward('aa', np.zeros(3), desc = "Rocket Angular Acceleration", unit = '1/s**2')
        
        #Kinematics transients
        self.add_transient('v', der = 'a', desc = "Rocket Velocity")
        self.add_transient('av', der = 'aa', desc = "Rocket Angular Velocity")
        self.add_transient('ar', der = 'av', desc = "Rocket Angular Position")
        
        #Kinematics outputs
        self.add_output(VelPort, 'v_out')
        self.add_outward('av_out', np.zeros(3), desc = "Rocket Angular Velocity", unit = '1/s')
        
    def compute(self):
        
        self.v_out.val = self.v
        self.av_out = self.av
        self.Kin_ang = self.ar
        


In [12]:
class Dynamics(System):
    
    def setup(self):
        
        #System orientation
        self.add_inward('Dyn_ang', np.zeros(3), desc = "Rocket Euler Angles", unit = '')
        
        #Kinematics inputs
        self.add_input(VelPort, 'v_in')
        self.add_inward('av_in', np.zeros(3), desc = "Rocket Angular Velocity", unit = '1/s')
        
        #Thrust inputs
        self.add_inward('Fp', np.zeros(3), desc = "Thrust Force", unit = 'N')
        self.add_inward('Mp', np.zeros(3), desc = "Thrust Moment", unit = 'N*m')
        
        #AeroForces inputs
        self.add_inward('Fa', np.zeros(3), desc = "Thrust Force", unit = 'N')
        self.add_inward('Ma', np.zeros(3), desc = "Thrust Moment", unit = 'N*m')
        
        #Geometry inputs
        self.add_inward('m', 100., desc = "Rocket Mass", unit = 'kg')
        self.add_inward('I', np.array([10., 100., 100.]), desc = "Rocket Moments of Inertia", unit = 'kg*m**2')
        
        #Gravity inputs
        self.add_input(AclPort, 'g')
        
        #Dynamics outputs
        self.add_outward('a', np.zeros(3), desc = "Rocket Acceleration", unit = 'm/s**2')
        self.add_outward('aa', np.zeros(3), desc = "Rocket Angular Acceleration", unit = '1/s**2')
        
    def compute(self):
        
        v = self.v_in.val
        av = self.av_in
        
        self.a[0] = (self.Fp[0] + self.Fa[0])/self.m + self.g.val[0] + av[2]*v[1] - av[1]*v[2]
        self.a[1] = (self.Fp[1] + self.Fa[1])/self.m + self.g.val[1] + av[0]*v[2] - av[2]*v[0]
        self.a[2] = (self.Fp[2] + self.Fa[2])/self.m + self.g.val[2] + av[1]*v[0] - av[0]*v[1]
        
        self.aa[0] = (self.Mp[0] + self.Ma[0] + (self.I[1] - self.I[2])*av[1]*av[2])/self.I[0]
        self.aa[1] = (self.Mp[1] + self.Ma[1] + (self.I[2] - self.I[0])*av[2]*av[0])/self.I[1]
        self.aa[2] = (self.Mp[2] + self.Ma[2] + (self.I[0] - self.I[1])*av[0]*av[1])/self.I[2]
        
        

In [13]:
class Thrust(System):
    
    def setup(self):
        
        #System constants
        self.add_inward('g0', 9.81, desc = "Gravity at Earth's Surface", unit = 'm/s**2')
        self.add_inward('isp', 100., desc = "Specific Impulsion in vacuum", unit = 's')
        
        #Rocket inputs
        self.add_inward('qp', 10., desc = "Engine's Propulsive Debt", unit = 'kg/s')
        
        #Pressure inputs
        self.add_inward('P', 100000., desc = "Atmospheric Pressure at Rocket's Height", unit = 'Pa')
        
        #Geometry inputs
        self.add_inward('S', 0.01, desc = "Propeller Area", unit = 'm**2')
        
        #Thrust outputs
        self.add_outward('Fp', np.zeros(3), desc = "Thrust Force", unit = 'N')
        self.add_outward('Mp', np.zeros(3), desc = "Thrust Moment", unit = 'N*m')
        
    def compute(self):
        
        self.Fp[0] = self.qp*self.g0*self.isp - self.P*self.S
        self.Fp[1] = 0.
        self.Fp[2] = 0.
        
        self.Mp[0] = 0.
        self.Mp[1] = 0.
        self.Mp[2] = 0.
        
        

In [14]:
class Geometry(System):
    
    def setup(self):
        
        #Geometry children
        self.add_child(Mass('Mass'), pulling = ['m_out', 'qp'])
        self.add_child(Inertia('Inertia'), pulling = ['I'])
        self.add_child(Dimensions('Dime'), pulling = ['S', 'S_ref'])
        self.add_child(CenterOfGravity('CoG'), pulling = ['gc'])
        
        #Children-children connections
        self.connect(self.Dime, self.CoG, ['l'])
        self.connect(self.Dime, self.Inertia, ['l', 'r_cyl'])
        self.connect(self.Mass, self.Inertia, {'m_out' : 'm_in'})
        
        #Execution order
        self.exec_order = ['Dime', 'Mass', 'Inertia', 'CoG']
    

In [15]:
class Aerodynamics(System):
    
    def setup(self):
        
        #System Orientation
        self.add_inward('Aero_ang', np.zeros(3), desc = "Rocket Euler Angles", unit = '')
        
        #Aerodynamics children
        self.add_child(AeroForces('AeroForces'), pulling = ['Fa', 'Ma', 'rho', 'S_ref', 'gc'])
        self.add_child(RelativeSpeed('RelSpeed'), pulling = ['v_in', 'V_wind'])
        self.add_child(Angles('Angles'))
        self.add_child(Coefficients('Coeff'))
        self.add_child(CenterOfPressure('CoP'))
        
        #Children-children connections
        self.connect(self.RelSpeed, self.Angles, ['V_rel'])
        self.connect(self.Angles, self.Coeff, ['alpha', 'beta'])
        self.connect(self.AeroForces, self.RelSpeed, ['V_rel'])
        self.connect(self.AeroForces, self.Coeff, ['C'])
        self.connect(self.AeroForces, self.CoP, ['gf'])
        
        
        #Execution order
        self.exec_order = ['RelSpeed', 'CoP', 'Angles', 'Coeff', 'AeroForces']
    

## Atmosphere Children

All vectors are given in the wind-fixed frame of reference

In [16]:
class Density(System):
    
    def setup(self):
        
        #System constants
        self.add_inward('rho0', 1.225, desc = "Air Density at Sea Level", unit = 'kg/m**3')
        
        #Trajectory inputs
        self.add_inward('r_in', np.zeros(3), desc = "Rocket's Position", unit = 'm')
        
        #Density outputs
        self.add_outward('rho', 1.225, desc = "Air Density at Rocket's Height", unit = 'kg/m**3')
        
    def compute(self):
        
        self.rho = self.rho0 * (20000. - self.r_in[-1]) / (20000. + self.r_in[-1])
        

In [17]:
class Pressure(System):
    
    def setup(self):
        
        #System constants
        self.add_inward('P0', 101325., desc = "Atmospheric Pressure at Sea Level", unit = 'Pa')
        
        #Trajectory inputs
        self.add_inward('r_in', np.zeros(3), desc = "Rocket's Position", unit = 'm')
        
        #Pressure outputs
        self.add_outward('P', 101325., desc = "Atmospheric Pressure at Rocket's Height", unit = 'Pa')
        
    def compute(self):
        
        self.P = self.P0 * (20000. - self.r_in[-1]) / (20000. + self.r_in[-1])


In [18]:
class Wind(System):
    
    def setup(self):
        
        #System orientation
        self.add_inward('Wind_ang', np.array([0., 0., np.pi/6]))
        
        #Wind inputs
        self.add_inward('V', 5., desc = "Wind Velocity", unit = 'm/s')
        
        #Wind outputs
        self.add_output(VelPort, 'V_wind')
        
    def compute(self):
        
        self.V_wind.val = np.array([self.V, 0., 0.])
        

## Geometry Children

In [19]:
class Mass(System):
    
    def setup(self):
        
        #System constants
        self.add_inward('m0', 100., desc = "Rocket's Initial Mass", unit = 'kg')
        self.add_inward('qp', 10., desc = "Engine's Propulsive Debt", unit = 'kg/s')
        self.add_inward('qi', 0., desc = "Engine's Inertial Debt", unit = 'kg/s')
        self.add_inward('dmdt', 0., desc = "Mass' Rate of Change", unit = 'kg/s')
        
        #System transients
        self.add_transient('m', der = "dmdt", desc = "Rocket Mass")
        
        self.add_outward('m_out', 100., desc = "Rocket Mass", unit = 'kg')
        
    def compute(self):
        
        self.dmdt = -(self.qp + self.qi)
        self.m_out = self.m
    
    

In [20]:
class Inertia(System):
    
    def setup(self):
        
        #Mass inputs
        self.add_inward('m_in', 100., desc = "Rocket Mass", unit = 'kg')
        
        #Dimensions inputs
        self.add_inward('l', 10., desc = "Rocket Length", unit = 'm')
        self.add_inward('r_cyl', 0.05, desc = "Rocket Radius", unit = 'm')
        
        #Inertia outputs
        self.add_outward('I', np.zeros(3), desc = "Rocket Moments of Inertia", unit = 'kg*m**2')
        
    def compute(self):
        
        self.I[0] = self.m_in*self.r_cyl**2/2
        self.I[1] = self.m_in*self.l**2/12
        self.I[2] = self.m_in*self.l**2/12
        
        

In [21]:
class Dimensions(System):
    
    def setup(self):
        
        #Dimensions outputs
        self.add_outward('l', 10., desc = "Rocket Length", unit = 'm')
        self.add_outward('r_cyl', 0.05, desc = "Rocket Radius", unit = 'm')
        self.add_outward('S', 0.01, desc = "Propeller Surface", unit = 'm**2')
        self.add_outward('S_ref', 1., desc = "Rocket Surface of Reference", unit = 'm**2')
        
    def compute(self):
        
        self.S_ref = np.pi*self.r_cyl**2
        

In [22]:
class CenterOfGravity(System):
    
    def setup(self):
        
        #Dimensions inputs
        self.add_inward('l', 10., desc = "Rocket Length", unit = 'm')
        
        #CenterOfGravity outputs
        self.add_outward('gc', np.zeros(3), desc = "GC Distance", unit = 'm')  #Distance between the propellers and the CoG
        
    def compute(self):
        
        self.gc = np.array([0., 0., self.l/2])
        

## Aerodynamics Children

All vectors are given in the velocity-fixed reference frame

In [23]:
class AeroForces(System):
    
    def setup(self):
        
        #Density inputs
        self.add_inward('rho', 1., desc = "Air Density", unit = 'kg/m**3')
        
        #Geometry inputs
        self.add_inward('S_ref', 1., desc = "Rocket Surface of Reference", unit = 'm**2')
        
        #RelativeSpeed inputs
        self.add_inward('V_rel', np.zeros(3), desc = "Relative Speed", unit = 'm/s')
        
        #Coefficients inputs
        self.add_inward('C', np.zeros(3), desc = "Aerodynamics Coefficients", unit = '')
        
        #CenterOfPressure inputs
        self.add_inward('gf', np.zeros(3), desc = "Rocket Center of Pressure", unit = 'm')
        
        #CenterOfGravity inputs
        self.add_inward('gc', np.zeros(3), desc = "Rocket Center of Mass", unit = 'm')
        
        #AeroForces outputs
        self.add_outward('Fa', np.zeros(3), desc = "Aerodynamics Forces", unit = 'N')
        self.add_outward('Ma', np.zeros(3), desc = "Aerodynamics Moments", unit = 'N*m')
        
    def compute(self):
        
        dl = self.gf - self.gc
        
        self.Fa[0] = -0.5*self.rho*self.S_ref*self.V_rel[0]**2*self.C[0]
        self.Fa[1] =  0.5*self.rho*self.S_ref*self.V_rel[1]**2*self.C[1]
        self.Fa[2] = -0.5*self.rho*self.S_ref*self.V_rel[2]**2*self.C[2]
        
        self.Ma[0] = 0.5*self.rho*self.S_ref*self.V_rel[0]**2*self.C[0]*dl[0]
        self.Ma[1] = 0.5*self.rho*self.S_ref*self.V_rel[1]**2*self.C[1]*dl[1]
        self.Ma[2] = 0.5*self.rho*self.S_ref*self.V_rel[2]**2*self.C[2]*dl[2]
    

In [24]:
class RelativeSpeed(System):
    
    def setup(self):
        
        #System orientation
        self.add_inward('RelSpeed_ang', np.zeros(3), desc = "Rocket Euler Angles", unit = '')
        
        #Kinematics inputs
        self.add_input(VelPort, 'v_in')
        
        #Wind inputs
        self.add_input(VelPort, 'V_wind')
        
        #RelativeSpeed outputs
        self.add_outward('V_rel', np.zeros(3), desc = "Relative Velocity", unit = 'm/s')
        
    def compute(self):
        
        self.V_rel = self.V_wind.val - self.v_in.val
        
    

In [25]:
class Angles(System):
    
    def setup(self):
        
        #RelativeSpeed inputs
        self.add_inward('V_rel', np.zeros(3), desc = "Relative Velocity", unit = 'm/s')
        
        #Angles outputs
        self.add_outward('alpha', 0., desc = "Angle of Attack", unit = '')
        self.add_outward('beta', 0., desc = "Angle of Bank", unit = '')
        
    def compute(self):
        
        self.alpha = np.arcsin(self.V_rel[2]/np.linalg.norm(self.V_rel))
        self.beta = np.arcsin(self.V_rel[1]/np.linalg.norm(self.V_rel))
        

In [26]:
class Coefficients(System):
    
    def setup(self):
        
        #System inputs
        self.add_inward('Cx0', 1., desc = "X Axis Initial Aerodynamic Coefficient", unit = '')
        self.add_inward('Cy0', 0., desc = "Y Axis Initial Aerodynamic Coefficient", unit = '')
        self.add_inward('Cz0', 0., desc = "Z Axis Initial Aerodynamic Coefficient", unit = '')
        
        self.add_inward('Cy_beta', 1., desc = "Cy Rate of Change", unit = '')
        self.add_inward('Cz_alpha', 1., desc = "Cz Rate of Change", unit = '')
        
        #Angles inputs
        self.add_inward('alpha', 0., desc = "Angle of Attack", unit = '')
        self.add_inward('beta', 0., desc = "Angle of Bank", unit = '')
        
        #Coefficients outputs
        self.add_outward('C', np.zeros(3), desc = "Aerodynamics Coefficient", unit = '')
        
    def compute(self):
        
        self.C[1] = self.Cy0 + self.Cy_beta*self.beta
        self.C[2] = self.Cz0 + self.Cz_alpha*self.alpha
        self.C[0] = self.Cx0 + 0.5*self.C[2]*self.alpha
    

In [27]:
class CenterOfPressure(System):
    
    def setup(self):
        
        #CenterOfPressure outputs
        self.add_outward('gf', np.array([0., 0., 8.]), desc = "GF Distance", unit = 'm')  #Distance between propellers and CoP
        
    

# Main Function

In [35]:
#Main function

#Time-step
dt = 0.1

#Create System
earth = Earth('earth')

#Add RungeKutta driver
driver = earth.add_driver(RungeKutta(order=4, dt=dt))
driver.time_interval = (0, 9)

#Add NonLinearSolver driver
solver = driver.add_child(NonLinearSolver('solver', factor=1))


# Add a recorder to capture time evolution in a dataframe
driver.add_recorder(
    DataFrameRecorder(includes=['Traj.r', 'Rocket.Kin.v', 'Rocket.Kin.a']),
    period=.1,
)

#Initial conditions and constants

driver.set_scenario(
    init = {
        'Traj.r' : np.zeros(3),
        'Rocket.Kin.v' : np.zeros(3),
        'Rocket.Kin.ar' : np.array([0., np.pi/2 + 1, 0.]),
        'Rocket.Kin.av' : 0.*np.array([np.pi/20, np.pi/20, 0.]),
        'Rocket.Geom.Mass.m' : 200.
    })


earth.run_drivers()

In [ ]:
data = driver.recorder.export_data()
data = data.drop(['Section', 'Status', 'Error code'], axis=1)
time = np.asarray(data['time'])
traj = np.asarray(data['Traj.r'].tolist())
velo = np.asarray(data['Rocket.Kin.v'].tolist())
acel = np.asarray(data['Rocket.Kin.a'].tolist())

In [ ]:
traj

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.86685070e-01,  9.21056539e-04,  6.70050504e-02],
       [-7.53035509e-01,  7.48576558e-03,  2.57098352e-01],
       [-1.70796899e+00,  2.56440884e-02,  5.53812739e-01],
       [-3.05970043e+00,  6.16643098e-02,  9.40615746e-01],
       [-4.81574254e+00,  1.22103424e-01,  1.40098498e+00],
       [-6.98289999e+00,  2.13786778e-01,  1.91846417e+00],
       [-9.56727273e+00,  3.43769054e-01,  2.47674326e+00],
       [-1.25742914e+01,  5.19330991e-01,  3.05972405e+00],
       [-1.60087351e+01,  7.47915130e-01,  3.65159871e+00],
       [-1.98747166e+01,  1.03715119e+00,  4.23690487e+00],
       [-2.41757462e+01,  1.39478828e+00,  4.80057936e+00],
       [-2.89147589e+01,  1.82866643e+00,  5.32809043e+00],
       [-3.40941721e+01,  2.34673636e+00,  5.80540054e+00],
       [-3.97158830e+01,  2.95693497e+00,  6.21913209e+00],
       [-4.57813286e+01,  3.66724606e+00,  6.55656633e+00],
       [-5.22915700e+01,  4.48563331e+00

In [ ]:
velo

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 3.87861100e+00,  1.47168576e-02,  8.60783007e-01],
       [ 7.77846068e+00,  6.09640085e-02,  1.79008361e+00],
       [ 1.16988647e+01,  1.41858608e-01,  2.78721795e+00],
       [ 1.56391342e+01,  2.60500894e-01,  3.85139026e+00],
       [ 1.95985611e+01,  4.19985937e-01,  4.98169245e+00],
       [ 2.35764417e+01,  6.23388208e-01,  6.17710057e+00],
       [ 2.75720555e+01,  8.73764556e-01,  7.43647956e+00],
       [ 3.15846801e+01,  1.17415736e+00,  8.75857974e+00],
       [ 3.56135782e+01,  1.52758197e+00,  1.01420432e+01],
       [ 3.96580127e+01,  1.93703379e+00,  1.15853890e+01],
       [ 4.37172281e+01,  2.40548466e+00,  1.30870375e+01],
       [ 4.77904704e+01,  2.93588289e+00,  1.46452956e+01],
       [ 5.18769651e+01,  3.53115361e+00,  1.62583566e+01],
       [ 5.59759348e+01,  4.19420465e+00,  1.79243064e+01],
       [ 6.00865969e+01,  4.92792301e+00,  1.96411198e+01],
       [ 6.42081395e+01,  5.73517517e+00

In [ ]:
acel

array([[ 3.86776612e+01, -7.87124549e-05,  8.26327715e+00],
       [ 3.88934575e+01,  2.99605631e-01,  8.95147288e+00],
       [ 3.91024283e+01,  6.30476004e-01,  9.63338863e+00],
       [ 3.93045401e+01,  9.92496277e-01,  1.03079719e+01],
       [ 3.94996666e+01,  1.38549730e+00,  1.09739717e+01],
       [ 3.96876849e+01,  1.80928432e+00,  1.16303274e+01],
       [ 3.98686717e+01,  2.26383719e+00,  1.22759787e+01],
       [ 4.00424109e+01,  2.74882324e+00,  1.29095675e+01],
       [ 4.02088849e+01,  3.26409141e+00,  1.35302249e+01],
       [ 4.03678796e+01,  3.80938211e+00,  1.41365897e+01],
       [ 4.05194774e+01,  4.38462373e+00,  1.47276933e+01],
       [ 4.06635609e+01,  4.98935113e+00,  1.53024660e+01],
       [ 4.08000105e+01,  5.62349393e+00,  1.58596419e+01],
       [ 4.09286041e+01,  6.28679126e+00,  1.63983446e+01],
       [ 4.10495055e+01,  6.97909270e+00,  1.69173025e+01],
       [ 4.11623783e+01,  7.70006803e+00,  1.74154380e+01],
       [ 4.12671690e+01,  8.44970290e+00

In [ ]:
import numpy as np
import plotly.graph_objects as go


x=[]
y=[]
z=[]

for i in range(len(traj)):
    x.append(traj[i][0])
    y.append(traj[i][1])
    z.append(traj[i][2])

fig = go.Figure(data=go.Scatter3d(
    x=x, y=y, z=z,
    marker=dict(
        size=4,
        color=z,
        colorscale='Viridis',
    ),
    line=dict(
        color='darkblue',
        width=2
    )
))


fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=2, range=[-750,750],),
        yaxis = dict(nticks=2, range=[-500,500],),
        zaxis = dict(nticks=2, range=[0,1500],),),
    width=700,
    margin=dict(r=20, l=10, b=10, t=10))


fig.show()

In [ ]:
earth

earth - Earth